## Predicting whether an email is spam:

### Stephen Phillips

In [132]:
import numpy as np
import pandas as pd
import nltk
import seaborn as sns
import sklearn.model_selection
import sklearn.preprocessing
pd.options.display.max_columns = 100
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV



In [133]:
cols = ['word_freq_make',
'word_freq_address',
 'word_freq_all',
 'word_freq_3d',
 'word_freq_our',
 'word_freq_over',
 'word_freq_remove',
 'word_freq_internet',
 'word_freq_order',
 'word_freq_mail',
 'word_freq_receive',
 'word_freq_will',
 'word_freq_people',
 'word_freq_report',
 'word_freq_addresses',
 'word_freq_free',
 'word_freq_business',
 'word_freq_email',
 'word_freq_you',
 'word_freq_credit',
 'word_freq_your',
 'word_freq_font',
 'word_freq_000',
 'word_freq_money',
 'word_freq_hp',
 'word_freq_hpl',
 'word_freq_george',
 'word_freq_650',
 'word_freq_lab',
 'word_freq_labs',
 'word_freq_telnet',
 'word_freq_857',
 'word_freq_data',
 'word_freq_415',
 'word_freq_85',
 'word_freq_technology',
 'word_freq_1999',
 'word_freq_parts',
 'word_freq_pm',
 'word_freq_direct',
 'word_freq_cs',
 'word_freq_meeting',
 'word_freq_original',
 'word_freq_project',
 'word_freq_re',
 'word_freq_edu',
 'word_freq_table',
 'word_freq_conference',
 'char_freq_;',
 'char_freq_(',
 'char_freq_[',
 'char_freq_!',
 'char_freq_$',
 'char_freq_#',
 'capital_run_length_average',
 'capital_run_length_longest',
 'capital_run_length_total',
 "spam"]

In [134]:
data = pd.read_csv("https://raw.githubusercontent.com/sphill12/Data_620/main/spambase.data", header = None, names = cols)


## EDA:

In [135]:
data.shape

(4601, 58)

In [136]:
data.describe()

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,word_freq_receive,word_freq_will,word_freq_people,word_freq_report,word_freq_addresses,word_freq_free,word_freq_business,word_freq_email,word_freq_you,word_freq_credit,word_freq_your,word_freq_font,word_freq_000,word_freq_money,word_freq_hp,word_freq_hpl,word_freq_george,word_freq_650,word_freq_lab,word_freq_labs,word_freq_telnet,word_freq_857,word_freq_data,word_freq_415,word_freq_85,word_freq_technology,word_freq_1999,word_freq_parts,word_freq_pm,word_freq_direct,word_freq_cs,word_freq_meeting,word_freq_original,word_freq_project,word_freq_re,word_freq_edu,word_freq_table,word_freq_conference,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total,spam
count,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000
mean,0.104553,0.213015,0.280656,0.065425,0.312223,0.095901,0.114208,0.105295,0.090067,0.239413,0.059824,0.541702,0.093930,0.058626,0.049205,0.248848,0.142586,0.184745,1.662100,0.085577,0.809761,0.121202,0.101645,0.094269,0.549504,0.265384,0.767305,0.124845,0.098915,0.102852,0.064753,0.047048,0.097229,0.047835,0.105412,0.097477,0.136953,0.013201,0.078629,0.064834,0.043667,0.132339,0.046099,0.079196,0.301224,0.179824,0.005444,0.031869,0.038575,0.139030,0.016976,0.269071,0.075811,0.044238,5.191515,52.172789,283.289285,0.394045
std,0.305358,1.290575,0.504143,1.395151,0.672513,0.273824,0.391441,0.401071,0.278616,0.644755,0.201545,0.861698,0.301036,0.335184,0.258843,0.825792,0.444055,0.531122,1.775481,0.509767,1.200810,1.025756,0.350286,0.442636,1.671349,0.886955,3.367292,0.538576,0.593327,0.456682,0.403393,0.328559,0.555907,0.329445,0.532260,0.402623,0.423451,0.220651,0.434672,0.349916,0.361205,0.766819,0.223812,0.621976,1.011687,0.911119,0.076274,0.285735,0.243471,0.270355,0.109394,0.815672,0.245882,0.429342,31.729449,194.891310,606.347851,0.488698
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.588000,6.000000,35.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.310000,0.000000,0.220000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00

In [137]:
data.head()

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,word_freq_receive,word_freq_will,word_freq_people,word_freq_report,word_freq_addresses,word_freq_free,word_freq_business,word_freq_email,word_freq_you,word_freq_credit,word_freq_your,word_freq_font,word_freq_000,word_freq_money,word_freq_hp,word_freq_hpl,word_freq_george,word_freq_650,word_freq_lab,word_freq_labs,word_freq_telnet,word_freq_857,word_freq_data,word_freq_415,word_freq_85,word_freq_technology,word_freq_1999,word_freq_parts,word_freq_pm,word_freq_direct,word_freq_cs,word_freq_meeting,word_freq_original,word_freq_project,word_freq_re,word_freq_edu,word_freq_table,word_freq_conference,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total,spam
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,0.00,0.64,0.00,0.00,0.00,0.32,0.00,1.29,1.93,0.00,0.96,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,0.21,0.79,0.65,0.21,0.14,0.14,0.07,0.28,3.47,0.00,1.59,0.0,0.43,0.43,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.07,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,0.38,0.45,0.12,0.00,1.75,0.06,0.06,1.03,1.36,0.32,0.51,0.0,1.16,0.06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.06,0.0,0.0,0.12,0.0,0.06,0.06,0.0,0.0,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,0.31,0.31,0.31,0.00,0.00,0.31,0.00,0.00,3.18,0.00,0.31,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,0.31,0.31,0.31,0.00,0.00,0.31,0.00,0.00,3.18,0.00,0.31,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1


## Split the model into train-test sets:

In [138]:
X = data.iloc[:,0:57]
y = data.iloc[:,-1]
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, train_size = 0.8)

## Feature Preprocessing:

In [139]:
scaler = sklearn.preprocessing.StandardScaler().fit(X_train)

In [140]:
x_train_transform = scaler.transform(X_train)
x_test_transform = scaler.transform(X_test)

## Model Testing:

### Random Forest

In [141]:
rf_classifier = RandomForestClassifier(n_estimators = 200)
rf_classifier.fit(x_train_transform, y_train)
y_pred = rf_classifier.predict(x_test_transform)
print("Model Accuracy:", metrics.accuracy_score(y_test,y_pred))
accuracy_df = pd.DataFrame({"model" : ["random forest, n = 200"],"accuracy": [metrics.accuracy_score(y_test,y_pred)]})

Model Accuracy: 0.9728555917480999


In [142]:

feature_imp = pd.Series(rf_classifier.feature_importances_, index = X_train.columns).sort_values(ascending= False)
feature_imp

char_freq_!                   0.134730
char_freq_$                   0.095403
word_freq_remove              0.075294
capital_run_length_average    0.056889
word_freq_your                0.055102
capital_run_length_longest    0.053364
word_freq_free                0.053024
word_freq_hp                  0.051070
capital_run_length_total      0.042570
word_freq_our                 0.039039
word_freq_money               0.038193
word_freq_you                 0.023785
word_freq_000                 0.023103
word_freq_george              0.020888
word_freq_edu                 0.019001
word_freq_hpl                 0.018102
word_freq_will                0.013166
word_freq_internet            0.012812
char_freq_(                   0.012509
word_freq_1999                0.012221
word_freq_all                 0.011273
word_freq_business            0.010624
word_freq_re                  0.010238
word_freq_receive             0.009760
word_freq_email               0.008978
word_freq_mail           

In [143]:
rf_classifier = RandomForestClassifier(n_estimators = 1000)
rf_classifier.fit(x_train_transform, y_train)
y_pred = rf_classifier.predict(x_test_transform)
accuracy_df.loc[len(accuracy_df.index)] = ["random forest, n = 1000", metrics.accuracy_score(y_test, y_pred)]
print("Model Accuracy:", metrics.accuracy_score(y_test,y_pred))


Model Accuracy: 0.9685124864277959


### Random Forest Using Tune Grid to calibrate hyperparameters

In [144]:
param_grid = {
    'n_estimators': [250, 500, 750, 1000],
    "max_features": ['sqrt', 'log2', None],
    "max_depth" : [3, 6, 9 ,None],
    "max_leaf_nodes" : [3,6,9, None]
}
grid_search = GridSearchCV(RandomForestClassifier(),
                           param_grid = param_grid)
grid_search.fit(x_train_transform, y_train)
print(grid_search.best_estimator_)

RandomForestClassifier(max_features='log2', n_estimators=250)


In [145]:
grid_tree_model = RandomForestClassifier(max_features = "log2", n_estimators = 1000)
grid_tree_model.fit(x_train_transform, y_train)
y_pred = grid_tree_model.predict(x_test_transform)
accuracy_df.loc[len(accuracy_df.index)] = ["grid search random forest", metrics.accuracy_score(y_test, y_pred)]
print("Model Accuracy:", metrics.accuracy_score(y_test,y_pred))

Model Accuracy: 0.9695982627578719


### Support Vector Machine Classification:

In [146]:
support_vector_model = SVC()
support_vector_model.fit(x_train_transform, y_train)
y_pred = support_vector_model.predict(x_test_transform)
print("Model Accuracy:", metrics.accuracy_score(y_test,y_pred))
accuracy_df.loc[len(accuracy_df.index)] = ["support vector", metrics.accuracy_score(y_test, y_pred)]

Model Accuracy: 0.9424538545059717


In [147]:
knn_model = KNeighborsClassifier()
knn_model.fit(x_train_transform, y_train)
y_pred = knn_model.predict(x_test_transform)
print("Model Accuracy:", metrics.accuracy_score(y_test, y_pred))
accuracy_df.loc[len(accuracy_df.index)] = ["k- nearest neighbor", metrics.accuracy_score(y_test, y_pred)]

Model Accuracy: 0.9087947882736156


KNN has the lowest accuracy of the three models

In [148]:
xgb_model = GradientBoostingClassifier()
xgb_model.fit(x_train_transform, y_train)
y_pred = xgb_model.predict(x_test_transform)
print("Model Accuracy:", metrics.accuracy_score(y_test, y_pred))
accuracy_df.loc[len(accuracy_df.index)] = ["gradient boost", metrics.accuracy_score(y_test, y_pred)]

Model Accuracy: 0.9641693811074918


In [149]:
feature_imp = pd.Series(xgb_model.feature_importances_, index = X_train.columns).sort_values(ascending= False)
feature_imp

char_freq_!                   0.290680
char_freq_$                   0.174426
word_freq_remove              0.121866
word_freq_hp                  0.089676
word_freq_free                0.054427
capital_run_length_average    0.051535
word_freq_your                0.039907
word_freq_george              0.031489
word_freq_edu                 0.031315
word_freq_our                 0.015993
capital_run_length_longest    0.015834
word_freq_money               0.014269
capital_run_length_total      0.013610
word_freq_meeting             0.006342
word_freq_650                 0.005803
word_freq_re                  0.004122
word_freq_internet            0.003725
word_freq_you                 0.003470
word_freq_000                 0.003348
word_freq_1999                0.003344
word_freq_will                0.003028
word_freq_email               0.003010
char_freq_(                   0.002912
word_freq_business            0.002475
word_freq_receive             0.002316
word_freq_report         

## Model Comparisons:

In [150]:
accuracy_df.sort_values(by = "accuracy", ascending= False)

,model,accuracy
0,"random forest, n = 200",0.972856
2,grid search random forest,0.969598
1,"random forest, n = 1000",0.968512
5,gradient boost,0.964169
3,support vector,0.942454
4,k- nearest neighbor,0.908795


## Conclusion:

A random forest model with hyperparameters that are highly calibrated will be the most effective model of those that were tested. All models perform well for the classification task though. Gradient Boost and the random forest model have about the same accuracy